# Pairs Trading Strategy

Pairs trading is a classic example of a strategy based on mathematical analysis. The principle is as follows. Let's say you have a pair of securities X and Y that have some underlying economic link. An example might be two companies that manufacture the same product, or two companies in one supply chain. If we can model this economic link with a mathematical model, we can make trades on it. We'll start by constructing a toy example.

Useful link: 
https://github.com/quantopian/research_public/blob/master/notebooks/lectures/Introduction_to_Pairs_Trading/notebook.ipynb
https://github.com/quantopian/research_public/blob/master/notebooks/lectures/Integration_Cointegration_and_Stationarity/notebook.ipynb

## **** Utils Functions ****
#### *Run this cell before the others, do not modify unless for specific needs*

In [1]:
pip install qstrader

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ib_insync

Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.stattools import coint
import statsmodels
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint, adfuller
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from datetime import datetime, timedelta
import itertools



## **** Asset Pairs and Data ****
To conduct our analysis, we are using minute-by-minute price data from January 1, 2024, to May 31, 2024.

In [4]:
pairs = [
    ('USDJPY=X', '^N225'),  # USD/JPY and Nikkei 225
    ('USDCAD=X', 'CL=F'),   # USD/CAD and Crude Oil Futures
    ('CL=F', 'USO'),        # Crude Oil Futures and United States Oil Fund
    ('GC=F', 'SI=F'),       # Gold Futures and Silver Futures
    ('AUDJPY=X', '^GSPC'),  # AUD/JPY and S&P 500
    ('USDCHF=X', 'GC=F'),   # USD/CHF and Gold Futures
    ('C', 'GS'),            # Citigroup and Goldman Sachs
    ('AAPL', 'META')        # Apple and Meta Platforms (formerly Facebook)
]

# Define the start and end dates for downloading data
start_date = '2024-01-01'
end_date = '2024-05-31'

## **** Choosing Cointegrated Assets with ADF and CADF Tests ****
Before implementing the strategy, it is crucial to select assets that exhibit
cointegration with one another. To achieve this, we utilize two models: the
Augmented Dickey-Fuller (ADF) test and the Cointegrated Augmented Dickey-
Fuller (CADF) test, as detailed below:

### ADF test:

The Augmented Dickey-Fuller (ADF) test is a statistical tool used to assess the stationarity of a time series. Stationarity is a key concept in time series analysis, indicating whether the statistical properties of the series, such as mean and variance, remain constant over time. Non-stationary series can display trends or seasonal patterns, potentially influencing the results of various time series models and analyses. We are conducting the ADF test on each asset to evaluate its stationarity, which is crucial for further analysis, including cointegration testing and the development of robust trading strategies. In this section, we will verify if each asset is non-stationary individually, as the concept of cointegration applies only to non-stationary time series.

In [5]:
# Function to perform the ADF test and print the results
def adf_test(series, name):
    series = series.dropna()
    result = adfuller(series)
    test_statistic = f'{result[0]:.3f}'
    p_value = f'{result[1]:.6f}'  # Display p-value with higher precision
    critical_values_str = ', '.join([f'{key}: {value:.3f}' for key, value in result[4].items()])

    # Determine if the series is stationary
    status = 'stationary' if result[1] < 0.05 else 'non-stationary'

    # Print the result
    print(f'{name} Test Statistic: {test_statistic}, p-value: {p_value}. Critical Values: {critical_values_str}. The series is likely {status}.')

# Extract unique tickers from pairs
unique_tickers = set(ticker for pair in pairs for ticker in pair)

# Loop through each ticker and perform the ADF test
for ticker in unique_tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval='1h')['Adj Close']
    adf_test(data, ticker)

[*********************100%***********************]  1 of 1 completed
GS Test Statistic: -0.462, p-value: 0.899206. Critical Values: 1%: -3.439, 5%: -2.866, 10%: -2.569. The series is likely non-stationary.
[*********************100%***********************]  1 of 1 completed
AAPL Test Statistic: -1.260, p-value: 0.647475. Critical Values: 1%: -3.440, 5%: -2.866, 10%: -2.569. The series is likely non-stationary.
[*********************100%***********************]  1 of 1 completed
^GSPC Test Statistic: -1.842, p-value: 0.359589. Critical Values: 1%: -3.440, 5%: -2.866, 10%: -2.569. The series is likely non-stationary.
[*********************100%***********************]  1 of 1 completed
AUDJPY=X Test Statistic: -0.416, p-value: 0.907395. Critical Values: 1%: -3.433, 5%: -2.863, 10%: -2.567. The series is likely non-stationary.
[*********************100%***********************]  1 of 1 completed
C Test Statistic: -1.463, p-value: 0.551565. Critical Values: 1%: -3.439, 5%: -2.866, 10%: -2.56

### CADF test:

The Cointegrated Augmented Dickey-Fuller (CADF) test is a statistical method used to determine whether two or more time series are cointegrated, implying a long-term equilibrium relationship despite being non-stationary on their own. In time series analysis, identifying cointegration is crucial because it allows for the modeling of long-term relationships between non-stationary series that share a common stochastic drift. We are conducting the CADF test on each pair of assets to assess their cointegration, which is fundamental for further analysis, such as developing pairs trading strategies. In this section, we will verify if each pair of assets is cointegrated, as cointegration analysis applies specifically to non-stationary time series that move together over time.

In [6]:
# Function to perform the CADF test and print the results
def cadf_test(series1, series2, name1, name2):
    series1 = series1.dropna()
    series2 = series2.dropna()

    # Ensure both series have the same length
    if len(series1) != len(series2):
        min_len = min(len(series1), len(series2))
        series1 = series1[-min_len:]
        series2 = series2[-min_len:]

    score, p_value, critical_values = coint(series1, series2)
    test_statistic = f'{score:.3f}'
    p_value_formatted = f'{p_value:.6f}'

    critical_values_str = ', '.join([f'{key}: {value:.3f}' for key, value in zip(['1%', '5%', '10%'], critical_values)])

    # Print critical values only once at the start
    if name1 == pairs[0][0] and name2 == pairs[0][1]:  # Assuming the first pair to print critical values
        print('======================================================')
        print(f'Critical Values: {critical_values_str}')
        print('======================================================')

    # Determine if the series are cointegrated
    status = 'cointegrated' if p_value < 0.05 else 'not cointegrated'

    # Print the result in one line
    print(f'{name1} & {name2} Test Statistic: {test_statistic}, p-value: {p_value_formatted}. The series are likely {status}.')

# Loop through each pair and perform the CADF test
for name1, name2 in pairs:
    # Download the data for each pair
    data1 = yf.download(name1, start=start_date, end=end_date)['Adj Close']
    data2 = yf.download(name2, start=start_date, end=end_date)['Adj Close']
    cadf_test(data1, data2, name1, name2)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Critical Values: 1%: -4.012, 5%: -3.399, 10%: -3.088
USDJPY=X & ^N225 Test Statistic: -0.781, p-value: 0.937120. The series are likely not cointegrated.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
USDCAD=X & CL=F Test Statistic: -2.649, p-value: 0.218499. The series are likely not cointegrated.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
CL=F & USO Test Statistic: -1.067, p-value: 0.889417. The series are likely not cointegrated.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
GC=F & SI=F Test Statistic: -0.726, p-value: 0.943716. The series are likely not cointegrated.
[******

Most of the pairs tested do not show significant evidence of cointegration, with the exception of the USD/CHF and Gold Futures pair, which demonstrates a strong long-term equilibrium relationship. This result is important for the development of trading strategies based on cointegration, as only cointegrated pairs offer opportunities to exploit such stable relationships. Therefore, for most of the pairs analyzed, other trading methodologies may be more appropriate.

# **** BACKTESTING ****
In this section, we will explore a backtesting using optimal timing strategies for trading a mean-reverting price spread. We'll optimize the positions of the previously identified pairs to ensure the intraday portfolio value aligns closely with an Ornstein-Uhlenbeck (OU) process, using maximum likelihood estimation for the best fit.


## Steps to Follow

1. **Download Data**
   - Download minute-level data for USD/CHF and GOLD from IBKR for the entire month of May.

2. **Estimate Parameters**
   - Estimate the parameters $\mu$ , $\theta$, $\sigma$, $B$, and the optimal exit level $b$ as of May 31, 2024.

3. **Conduct Backtesting**
   - Conduct backtesting for the remaining days up to the current date to validate the strategy.


In [27]:
import nest_asyncio
from ib_insync import *
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import datetime as dt
import time

nest_asyncio.apply()

# Connect to IBKR TWS or Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=2)

# Define the contracts for USD/CHF (Forex) and GC (Gold Futures)
usdchf_contract = Forex('USDCHF')
gc_contract = Future('GC', '202406', 'COMEX')

# Function to fetch historical data
def fetch_month_data(contract, start_date='20240501 00:00:00', end_date='20240531 23:59:59', bar_size='1 min'):
    df_list = []
    current_end_date = end_date

    while True:
        try:
            bars = ib.reqHistoricalData(
                contract,
                endDateTime=current_end_date,
                durationStr='1 D',  # Fetch 1 day of data at a time
                barSizeSetting=bar_size,
                whatToShow='MIDPOINT' if isinstance(contract, Forex) else 'TRADES',
                useRTH=False,
                formatDate=1
            )
            if not bars:
                break
            df = util.df(bars)
            df_list.append(df)
            current_end_date = df['date'].iloc[0]
            time.sleep(10)
            if dt.datetime.strptime(current_end_date, '%Y-%m-%d %H:%M:%S') <= dt.datetime.strptime(start_date, '%Y%m%d %H:%M:%S'):
                break
        except Exception as e:
            print(f"Error fetching data: {e}")
            break

    return pd.concat(df_list) if df_list else None

# Fetch data for USD/CHF and Gold
usdchf_data = fetch_month_data(usdchf_contract)
gc_data = fetch_month_data(gc_contract)

# Disconnect from IBKR
ib.disconnect()

# Save data to CSV
usdchf_data.to_csv('usdchf_may_2024_data.csv', index=False)
gc_data.to_csv('gc_may_2024_data.csv', index=False)

# Load data from CSV files
usdchf_data = pd.read_csv('usdchf_may_2024_data.csv')
gc_data = pd.read_csv('gc_may_2024_data.csv')

# Convert the 'date' column to datetime and set as index
usdchf_data['date'] = pd.to_datetime(usdchf_data['date'])
gc_data['date'] = pd.to_datetime(gc_data['date'])
usdchf_data.set_index('date', inplace=True)
gc_data.set_index('date', inplace=True)

# Resample to 1-hour intervals
usdchf_resampled = usdchf_data.resample('1H').last().dropna()
gc_resampled = gc_data.resample('1H').last().dropna()

# Merge datasets
merged_data = pd.merge(usdchf_resampled, gc_resampled, left_index=True, right_index=True, suffixes=('_usdchf', '_gc'))
merged_data.dropna(inplace=True)

# Extract price arrays
usdchf_prices = merged_data['close_usdchf'].values
gc_prices = merged_data['close_gc'].values

# Function to construct portfolio
def construct_portfolio(usdchf_prices, gc_prices, B):
    return usdchf_prices - B * gc_prices

# Function to optimize pair ratio
def optimize_pair_ratio(usdchf_prices, gc_prices):
    def negative_log_likelihood(B):
        portfolio_values = construct_portfolio(usdchf_prices, gc_prices, B)
        portfolio_values = (portfolio_values - np.mean(portfolio_values)) / np.std(portfolio_values)
        _, theta, _ = optimize_ou_parameters(portfolio_values)
        return -theta
    result = minimize(negative_log_likelihood, 0.0, bounds=[(-2, 2)])
    B_opt = result.x[0]
    return B_opt

# Function to optimize OU parameters
def optimize_ou_parameters(portfolio_values):
    def ou_log_likelihood(params, X):
        mu, theta, sigma = params
        n = len(X)
        dt = 1
        X_diff = np.diff(X)
        X_prev = X[:-1]
        likelihood = (
            -n / 2 * np.log(2 * np.pi * sigma**2 * (1 - np.exp(-2 * mu * dt)) / (2 * mu))
            - (X_diff - (theta - X_prev) * (1 - np.exp(-mu * dt)))**2
            / (2 * sigma**2 * (1 - np.exp(-2 * mu * dt)) / (2 * mu))
        )
        return -np.sum(likelihood)
    initial_params = [0.1, portfolio_values.mean(), portfolio_values.std()]
    bounds = [(1e-6, None), (None, None), (1e-6, None)]
    result = minimize(ou_log_likelihood, initial_params, args=(portfolio_values,), bounds=bounds)
    return result.x

# Function to calculate optimal exit level
def calculate_optimal_exit_level(theta, mu, sigma, c=0):
    def F(x):
        integral_result, _ = quad(
            lambda u: (u**(2*mu/sigma**2 - 1)) * np.exp(np.sqrt(2*mu/sigma**2) * (theta - x) * u - u**2 / 2), 
            0, 
            np.inf
        )
        return integral_result
    
    def F_prime(x):
        integral_result, _ = quad(
            lambda u: (u**(2*mu/sigma**2 - 2)) * np.exp(np.sqrt(2*mu/sigma**2) * (theta - x) * u - u**2 / 2) 
            * (np.sqrt(2*mu/sigma**2) * (theta - x) - u), 
            0, 
            np.inf
        )
        return integral_result
    
    def objective(b):
        return np.abs(F(b) - (b - c) * F_prime(b))
    
    result = minimize(objective, theta, bounds=[(theta - 5 * sigma, theta + 5 * sigma)])
    return result.x[0]

# Function to calculate signals for optimal exit model
def calculate_optimal_exit_signals(portfolio_values, optimal_exit_level, window_size=30):
    signals = []
    for i in range(window_size, len(portfolio_values)):
        current_value = portfolio_values[i]
        moving_avg = np.mean(portfolio_values[i - window_size:i])
        std_dev = np.std(portfolio_values[i - window_size:i])
        if current_value < moving_avg - std_dev:
            signals.append(1)
        elif current_value > optimal_exit_level:
            signals.append(-1)
        else:
            signals.append(0)
    signals = [0] * window_size + signals
    return np.array(signals)

# Function to simulate trading strategy
def simulate_optimal_exit_strategy(portfolio_values, signals, initial_cash=10000):
    position = 0
    cash = initial_cash
    portfolio = []
    min_length = min(len(signals), len(portfolio_values))
    for i in range(min_length):
        signal = signals[i]
        current_value = portfolio_values[i]
        if signal == 1 and position == 0:
            position = 1
            cash -= current_value
        elif signal == -1 and position == 1:
            position = 0
            cash += current_value
        portfolio_value = cash + (position * current_value)
        portfolio.append(portfolio_value)
    return np.array(portfolio)

# Optimize pair ratio
B_opt = optimize_pair_ratio(usdchf_prices, gc_prices)
print(f"Optimized pair ratio B: {B_opt:.4f}")

# Construct the portfolio
portfolio_values = construct_portfolio(usdchf_prices, gc_prices, B_opt)

# Optimize the OU parameters
mu_opt, theta_opt, sigma_opt = optimize_ou_parameters(portfolio_values)
print(f"Optimized parameters: mu={mu_opt:.4f}, theta={theta_opt:.4f}, sigma={sigma_opt:.4f}")

# Calculate the optimal exit level
optimal_exit_level = calculate_optimal_exit_level(theta_opt, mu_opt, sigma_opt)
print(f"Optimal exit level b: {optimal_exit_level:.4f}")

# Calculate the optimal exit signals
optimal_exit_signals = calculate_optimal_exit_signals(portfolio_values, optimal_exit_level)

# Simulate the optimal exit strategy
portfolio_simulation = simulate_optimal_exit_strategy(portfolio_values, optimal_exit_signals)

# Print the final portfolio value
final_value = portfolio_simulation[-1]
print(f"Final portfolio value: ${final_value:.2f}")



Error fetching data: strptime() argument 1 must be str, not Timestamp
Error fetching data: strptime() argument 1 must be str, not Timestamp
Optimized pair ratio B: 0.0744
Optimized parameters: mu=0.1789, theta=-172.8241, sigma=0.0819
Optimal exit level b: -172.4148
Final portfolio value: $10000.00


# **** LIVE TRADING ****

## Steps to Follow

1. **Download Data**
   - Obtain minute-level data for USD/CHF and GOLD from Interactive Brokers (IBKR) for the past 30 days.

2. **Estimate Parameters**
   - Calculate the parameters $\mu$, $\theta$, $\sigma$, $B$, and the optimal exit level $b$ using the most recent 30 days of data, as of today.

3. **Connect to IBKR - real time data**
   - Establish a connection to IBKR to access real-time market data.

4. **Live trading execution**
   - Initiate the trading algorithm to begin executing the strategy with live data.


#### 1. Download Data

In [16]:
# Import necessary libraries
import nest_asyncio
import asyncio
from ib_insync import *

# Import other useful libraries
import pandas as pd

# Allow nested event loops (needed for Jupyter Notebooks)
nest_asyncio.apply()

# Connect to IBKR TWS or Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=2)

# Define the contract for USD/CHF (Forex)
usdchf_contract = Forex('USDCHF')

# Define the contract for GC (Gold Futures)
gc_contract = Future('GC', '202406', 'COMEX')  # Update '202406' with the relevant expiry date

# Function to fetch historical data
def fetch_historical_data(contract, duration='30 D', bar_size='1 min'):
    """Fetch historical data for a given contract."""
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow='MIDPOINT' if isinstance(contract, Forex) else 'TRADES',
        useRTH=True
    )
    if bars:
        df = util.df(bars)
        print(f"Historical data fetched for {contract.symbol}:")
        print(df.head())
        return df
    else:
        print(f"No historical data fetched for {contract.symbol}")
        return None

# Fetch and display data for USD/CHF
usdchf_data = fetch_historical_data(usdchf_contract, duration='30 D', bar_size='1 min')

# Fetch and display data for Gold Futures
gc_data = fetch_historical_data(gc_contract, duration='30 D', bar_size='1 min')

# Disconnect from IBKR
ib.disconnect()

# Save the data to CSV for further analysis
if usdchf_data is not None:
    usdchf_data.to_csv('usdchf_data.csv', index=False)
if gc_data is not None:
    gc_data.to_csv('gc_data.csv', index=False)


reqHistoricalData: Timeout for Forex('USDCHF', exchange='IDEALPRO')
Error 162, reqId 3: Messaggio di errore servizio dati di mercato storici:API historical data query cancelled: 3, contract: Forex('USDCHF', exchange='IDEALPRO')


No historical data fetched for USD
Historical data fetched for GC:
                       date    open    high     low   close  volume  average  \
0 2024-05-07 09:30:00-04:00  2326.5  2326.8  2325.7  2326.1   303.0  2326.20   
1 2024-05-07 09:31:00-04:00  2326.0  2326.1  2325.1  2325.2   242.0  2325.63   
2 2024-05-07 09:32:00-04:00  2325.2  2327.1  2325.1  2326.8   501.0  2326.09   
3 2024-05-07 09:33:00-04:00  2326.7  2327.9  2325.6  2327.5  1550.0  2327.04   
4 2024-05-07 09:34:00-04:00  2327.3  2327.7  2327.0  2327.0   256.0  2327.33   

   barCount  
0       195  
1       156  
2       265  
3       689  
4       150  


In [17]:
# Load data
usdchf_data = pd.read_csv('usdchf_data.csv')
gc_data = pd.read_csv('gc_data.csv')

# Merge data on datetime
usdchf_data['date'] = pd.to_datetime(usdchf_data['date'])
gc_data['date'] = pd.to_datetime(gc_data['date'])

# Resample to 1-hour bars and merge
usdchf_data.set_index('date', inplace=True)
gc_data.set_index('date', inplace=True)
usdchf_resampled = usdchf_data.resample('1H').last().dropna()
gc_resampled = gc_data.resample('1H').last().dropna()

# Align the data
merged_data = pd.merge(usdchf_resampled, gc_resampled, left_index=True, right_index=True, suffixes=('_usdchf', '_gc'))
merged_data.dropna(inplace=True)

#### 2. Estimate parameters

In [18]:
# Construct the portfolio
def construct_portfolio(usdchf_prices, gc_prices, B):
    """Construct the portfolio value."""
    return usdchf_prices - B * gc_prices


# Define the OU log-likelihood function
def ou_log_likelihood(params, X):
    mu, theta, sigma = params
    n = len(X)
    dt = 1  # Assuming data is resampled to hourly
    X_diff = np.diff(X)
    X_prev = X[:-1]
    likelihood = (
        -n / 2 * np.log(2 * np.pi * sigma**2 * (1 - np.exp(-2 * mu * dt)) / (2 * mu))
        - (X_diff - (theta - X_prev) * (1 - np.exp(-mu * dt)))**2
        / (2 * sigma**2 * (1 - np.exp(-2 * mu * dt)) / (2 * mu))
    )
    return -np.sum(likelihood)

# Optimize parameters for OU process
def optimize_ou_parameters(portfolio_values):
    """Optimize the parameters for the OU process."""
    initial_params = [0.1, portfolio_values.mean(), portfolio_values.std()]
    bounds = [(1e-6, None), (None, None), (1e-6, None)]
    result = minimize(ou_log_likelihood, initial_params, args=(portfolio_values,), bounds=bounds)
    mu_opt, theta_opt, sigma_opt = result.x
    print(f"Optimized parameters: mu={mu_opt:.4f}, theta={theta_opt:.4f}, sigma={sigma_opt:.4f}")
    return mu_opt, theta_opt, sigma_opt

# Function to optimize the pair ratio
def optimize_pair_ratio(usdchf_prices, gc_prices):
    """Optimize the pair ratio B."""
    def negative_log_likelihood(B):
        portfolio_values = construct_portfolio(usdchf_prices, gc_prices, B)
        portfolio_values = np.array(portfolio_values)
        portfolio_values = (portfolio_values - np.mean(portfolio_values)) / np.std(portfolio_values)
        _, theta, _ = optimize_ou_parameters(portfolio_values)
        return -theta  # We want to maximize mean-reversion (i.e., mean is closer to zero)

    result = minimize(negative_log_likelihood, 0.0, bounds=[(-2, 2)])  # Adjust bounds as necessary
    B_opt = result.x[0]
    print(f"Optimized pair ratio B: {B_opt:.4f}")
    return B_opt

In [19]:
# Run the optimization
usdchf_prices = merged_data['close_usdchf'].values
gc_prices = merged_data['close_gc'].values

# Optimize pair ratio
B_opt = optimize_pair_ratio(usdchf_prices, gc_prices)

# Construct portfolio with optimized pair ratio
portfolio_values = construct_portfolio(usdchf_prices, gc_prices, B_opt)

# Optimize OU parameters for the constructed portfolio
optimize_ou_parameters(portfolio_values)

Optimized parameters: mu=0.0480, theta=-0.5134, sigma=0.0466
Optimized parameters: mu=0.0480, theta=-0.5133, sigma=0.0466
Optimized parameters: mu=0.1602, theta=0.1527, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1527, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1527, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1527, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1526, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1526, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta=0.1522, sigma=0.0613
Optimized parameters: mu=0.1602, theta

(0.16019019869433226, -590.8838873628364, 0.3189628780197693)

In [20]:
import scipy.integrate as integrate

# Function to calculate the optimal exit level
def calculate_optimal_exit_level(theta, mu, sigma, c=0):
    """Calculate the optimal exit level b."""
    def F(x):
        integral_result, _ = integrate.quad(
            lambda u: (u**(2*mu/sigma**2 - 1)) * np.exp(np.sqrt(2*mu/sigma**2) * (theta - x) * u - u**2 / 2), 
            0, 
            np.inf
        )
        return integral_result
    
    def F_prime(x):
        integral_result, _ = integrate.quad(
            lambda u: (u**(2*mu/sigma**2 - 2)) * np.exp(np.sqrt(2*mu/sigma**2) * (theta - x) * u - u**2 / 2) 
            * (np.sqrt(2*mu/sigma**2) * (theta - x) - u), 
            0, 
            np.inf
        )
        return integral_result
    
    def objective(b):
        return np.abs(F(b) - (b - c) * F_prime(b))
    
    result = minimize(objective, theta, bounds=[(theta - 5 * sigma, theta + 5 * sigma)])
    b_opt = result.x[0]
    print(f"Optimal exit level b: {b_opt:.4f}")
    return b_opt

# Calculate the optimal exit level
mu_opt, theta_opt, sigma_opt = optimize_ou_parameters(portfolio_values)
optimal_exit_level = calculate_optimal_exit_level(theta_opt, mu_opt, sigma_opt)


Optimized parameters: mu=0.1602, theta=-590.8839, sigma=0.3190
Optimal exit level b: -589.2891


Now we calclate the optimal exit model:

In [21]:
import numpy as np

# Function to calculate signals for optimal exit model
def calculate_optimal_exit_signals(portfolio_values, optimal_exit_level, window_size=30):
    """Calculate signals for the optimal exit model."""
    signals = []
    for i in range(window_size, len(portfolio_values)):
        current_value = portfolio_values[i]
        moving_avg = np.mean(portfolio_values[i - window_size:i])
        std_dev = np.std(portfolio_values[i - window_size:i])

        # Entry signal: enter when portfolio value is below the lower band
        if current_value < moving_avg - std_dev:
            signals.append(1)  # Buy signal
        # Exit signal: exit when portfolio value is above the optimal exit level
        elif current_value > optimal_exit_level:
            signals.append(-1)  # Sell signal
        else:
            signals.append(0)  # Hold

    return np.array(signals)


#### 3. Connect to IBKR - real time data

In [37]:
import nest_asyncio
from ib_insync import IB, Forex, Future, util
import pandas as pd
import numpy a
from scipy.optimize import minimize
import scipy.integrate as integrate
import time

# Allow nested event loops (needed for Jupyter Notebooks)
nest_asyncio.apply()


In [ ]:
# Function to connect to IBKR
def connect_ibkr(ib, host='127.0.0.1', port=7497, client_id=2):
    """Connect to IBKR TWS or Gateway."""
    if not ib.isConnected():
        ib.connect(host, port, clientId=client_id)
    else:
        print("Already connected to IBKR.")
    return ib

# Connect to IBKR TWS or Gateway
ib = IB()
ib = connect_ibkr(ib)

# Check connection status
if not ib.isConnected():
    print("Not connected to IBKR. Exiting...")
    exit()


Contract definition:

In [38]:
# Define the contracts
usdchf_contract = Forex('USDCHF')
gc_contract = Future('GC', '202406', 'COMEX')  # Update '202406' with the relevant expiry date


Fetching Real-Time Data:

In [39]:
# Function to fetch real-time data
def fetch_realtime_data(contract, duration='10 D', bar_size='1 hour'):
    """Fetch real-time data for a given contract."""
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow='MIDPOINT' if isinstance(contract, Forex) else 'TRADES',
        useRTH=True,
        formatDate=1,
        keepUpToDate=True
    )
    return bars

# Fetch initial data
usdchf_bars = fetch_realtime_data(usdchf_contract)
gc_bars = fetch_realtime_data(gc_contract)


ConnectionError: Not connected

Order Placement:

In [ ]:
# Function to place order on IBKR
def place_order(contract, action, quantity):
    """Place an order with IBKR."""
    order = MarketOrder(action, quantity)
    trade = ib.placeOrder(contract, order)
    return trade


#### 4. Live trading execution

In [ ]:
# Initialize variables for live trading
B_opt = None
optimal_exit_level = None
current_position = 0  # 0: no position, 1: long position

# Function to update portfolio and execute strategy
def execute_live_strategy():
    global B_opt, optimal_exit_level, current_position

    # Update data
    usdchf_prices = np.array([bar.close for bar in usdchf_bars])
    gc_prices = np.array([bar.close for bar in gc_bars])

    if len(usdchf_prices) < 30 or len(gc_prices) < 30:
        print("Not enough data to compute strategy. Waiting for more data.")
        return

    # Optimize pair ratio
    B_opt = optimize_pair_ratio(usdchf_prices, gc_prices)

    # Construct portfolio
    portfolio_values = construct_portfolio(usdchf_prices, gc_prices, B_opt)

    # Optimize OU parameters
    mu_opt, theta_opt, sigma_opt = optimize_ou_parameters(portfolio_values)

    # Calculate optimal exit level
    optimal_exit_level = calculate_optimal_exit_level(theta_opt, mu_opt, sigma_opt)

    # Generate trading signals
    signals = calculate_optimal_exit_signals(portfolio_values, optimal_exit_level)

    # Execute trading strategy
    last_signal = signals[-1]
    if last_signal == 1 and current_position == 0:  # Buy signal and no current position
        print("Placing buy


#### *References *
Hull, J.C. (2017) Option, Futures, and Other Derivatives (Pearson), 9th edition

Leung T., Xin L. (2015) Optimal Mean Reversion Trading with Transaction Costs and Stop-Loss Exit

Lee D., Leung T. (2020) On the E cacy of Optimized Exit Rule for Mean Reversion Trading